In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display

import os

# Keras functions
import tensorflow as tf

from keras.optimizers import Adam
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten
from keras.layers import Dropout, LeakyReLU, BatchNormalization
from keras.layers import Activation, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Reshape
from matplotlib import pyplot
from IPython.display import clear_output

# Numpy functions
import numpy
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from numpy import zeros
from numpy import ones
from numpy import asarray

#Torchvision for fast and easy loading and resizing
import torchvision
import torchvision.transforms as transforms

from PIL import Image

#for efficient data loading and manipulation, the images are save as binary file
#to reproduce the file the  code is below
#but if the saved file is already present the function will load it and not
#create it from scratch

In [ ]:
#change these paths accordingly
dataset_path = "../input/best-artworks-of-all-time/resized" 
saved_binary_file = "/kaggle/working/training_data.npy"

In [ ]:
WIDTH = 64
HEIGHT = 64
IMG_SIZE = (WIDTH,HEIGHT)
EPOCHS = 250

In [ ]:
def prepare_data(path_of_file , path_of_data):
    #Look for saved file to save time loading and processing images between runs
    print("Looking for saved binary file...")

    if not os.path.isfile(path_of_file):
        print("\n File not found, creating  new file...\n")
        dataset = []
        transform_ds = transforms.Compose([transforms.Resize(IMG_SIZE),]) #define transformation
        
        image_folder = torchvision.datasets.ImageFolder(root=path_of_data,
                                        transform=transform_ds)

        print('Number of artworks found: ',len(image_folder))
        
        
        print("Converting images, this will take a few minutes")
        for i in range (len(image_folder)):
            image_array = numpy.array(image_folder[i][0])
            dataset.append(image_array)
            if (i%500 == 0):
                print("Pictures processed: ", i)
                
        print("Saving dataset binary file...")
        dataset = np.array(dataset, dtype=np.float32)
        dataset = (dataset - 127.5) / 127.5 #Normalize to [-1 , 1]
        numpy.save(path_of_file, dataset)  #Save processed images as npy file

    else:
        print("Data found, loading..")
        dataset = np.load(path_of_file) 

    print("Dataset length: ", len(dataset))

    return dataset

dataset = prepare_data(saved_binary_file , dataset_path)

#Using a TensorFlow Dataset to manage the images for easy shuffling, dividing etc
BATCH_SIZE = 128

training_dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(60000).batch(BATCH_SIZE)

In [ ]:
def visualize_random_art(dataset):
    """
    function to plot some random images from the data set 

    """
    np.random.shuffle(dataset) #Shuffle the images

    fig = plt.figure(figsize=(12,12))
    for i in range(1,37):
        fig.add_subplot(6,6,i)
        plt.imshow(dataset[i])
        plt.axis('off')

visualize_random_art(dataset)

In [ ]:
def make_generator_model(seed_size, channels):
    """
    This function builds the generator for DCGAN 
    Parameters:
        seed_size:according to authors of DCGAN , Generator takes this random seed and generates an image
        channels : output channels that image will have
    """
    model = tf.keras.Sequential()
    
    model.add(Dense(4*4*512,activation="relu",input_dim=seed_size)) #64x64 units
    model.add(Reshape((4,4,512)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))


    model.add(Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha =0.3))
    model.add(Dropout(0.4))

    model.add(Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dropout(0.4))

    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha =0.3))
    model.add(Dropout(0.4))
    
    model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False  ))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2DTranspose(channels, (5, 5), strides=(1, 1), padding='same', use_bias=True, activation='tanh'))

 

    return model

In [ ]:
def make_discriminator_model(image_shape):
    """
    This function builds a discriminator which is a CNN based image classifier
    and will output probability values of what it thinks is fake or real with  values close to
    0 being fake and 1 being real.

    Parameters :
        image_shape : input_image shape (h x w x c)
    """
    model = tf.keras.Sequential()
    model.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(64, 64, 3), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(256, kernel_size=5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(512, kernel_size=5, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    return model

In [ ]:
SEED_SIZE = 100
IMAGE_CHANNELS = 3
noise = tf.random.normal([1,SEED_SIZE])
image_shape = (HEIGHT, HEIGHT, IMAGE_CHANNELS)
cross_entropy = tf.keras.losses.BinaryCrossentropy()


#uncomment these lines only if model is to be trained from scratch

generator = make_generator_model(SEED_SIZE ,IMAGE_CHANNELS)
generated_image = generator(noise, training=False) # untrained generator output
plt.imshow(generated_image[0, :, :, 0])
discriminator = make_discriminator_model(image_shape) #untrained discriminator output
print(discriminator(generated_image))

In [ ]:
def discriminator_loss(real_output, fake_output):
    """
    The discriminators loss is based on its ability to distinguish real images from fakes.
    It compares its predictions on real images to an array of ones (remember 1 being real)
    and its predictions on fake images to an array of zeros (0 being fake). 
    The goal is to classify all real images as 1 and all fakes as 0. 
    The total loss is then these two losses added together.

    Parameters : 
        real_output  : real image from the dataset 
        fake_output  : fake image from the dataset
    """
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    """
    The generators loss is a measurement of how good it performed at fooling the discriminator. 
    If the discriminator classifies the fake images as 1, the generator did a good job.

    Parameters :
     fake_output : fake image from generator
    """
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
#The two models optimizers are separated because we train them separately.
# I found a slightly lower generator LR to be beneficial.
#Beta value of 0.5 generates more stable models as per the findings in the paper 
#"Unsupervised representation learning with deep convolutional generative adversarial networks"

generator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)

In [ ]:
!mkdir ./training_checkpoints

#checkpoint for saving a model

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [ ]:
def generate_images(generated_images2):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
    generated_images2 =0.5 * generated_images2 + 0.5

    fig = plt.figure(figsize=(10,10))
    for i in range(1,21):
        fig.add_subplot(5,5,i)
        plt.imshow(generated_images2[i])
        plt.axis('off')
    plt.savefig("Images.png")
    plt.show()

In [ ]:
EPOCHS = 150
noise_dim = 100
num_examples_to_generate = 16

# Reuse this seed overtime
# to visualize progress in the animated GIF
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    """
    The training begins by providing a random seed to the generator, which is then used to generate
    an image. The discriminator then classifies images from both the fake and real dataset.
    The loss is calculated separately for each model and the gradients are updated.

    Parameters  :
    images : images to be trained on
    """
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      #generate_images(generated_images)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss , disc_loss

def train(dataset, epochs):
  for epoch in range(epochs):
    gen_loss_list = []
    disc_loss_list = []
    start = time.time()

    for image_batch in dataset:
        t =  train_step(image_batch)
        gen_loss_list.append(t[0])
        disc_loss_list.append(t[1])
    # Produce images for the GIF as you go
    # display.clear_output(wait=True)
    g_loss = sum(gen_loss_list) / len(gen_loss_list) #calculate losses
    d_loss = sum(disc_loss_list) / len(disc_loss_list)
    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
    
    print(f'Epoch {epoch+1}, gen loss = {g_loss}, disc loss = {d_loss}')
  # Generate after the final epoch
  display.clear_output(wait=True)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
train(training_dataset, EPOCHS) # uncomment to train model from scratch or from a previous checkpoint

In [ ]:
def save_model():
    generator.save("trained_generator.h5")
save_model() #uncomment to save a newly trained model

In [ ]:
def load_model(path_of_saved_h5_file):
    import keras
    generator = keras.models.load_model(path_of_saved_h5_file)

load_model("./trained_generator.h5")

In [ ]:
noise = tf.random.normal([22,SEED_SIZE])
generated_image = generator(noise, training=False)
generate_images(generated_image)